In [1]:
from keras.models import Sequential
from keras.layers import Dense
import sklearn
import numpy as np

In [ ]:
class CommitteeMember:
    def __init__(self):
        self.model = Sequential()
        self.model.add(Dense(128, input_dim=768*2, activation='relu'))
        self.model.add(Dense(2, activation='softmax'))

        # Compile the model
        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train, epochs=25, batch_size=64)

In [ ]:
neural_net = Sequential()
neural_net.add(Dense(256, input_dim=768*2, activation='relu'))
neural_net.add(Dense(256, activation='relu'))
neural_net.add(Dense(2, activation='softmax'))

neural_net.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#reset training set and pool
committee = [CommitteeMember() for _ in range(10)]
testacc_qbc=[] #this should hold the final accuracies
trainset = None # should be labeled training data
Xtrain = trainset['embedding'] # should be the concatinated embeddings for question and context
ytrain = trainset['label']
Xpool = None # unlabeled data in pool
n_samples = 1000 # number of samples to be labeled from pool
n_add = 10
#poolidx=np.arange(len(Xpool),dtype=np.int64)
#poolidx=np.setdiff1d(poolidx,trainset)
for i in range(n_samples):
    ypool_lab = []
    # fit models in committee
    for committee_member in committee:
        #bootstrapping
        Xtr,ytr=sklearn.utils.resample(Xtrain,ytrain,stratify=ytrain)
        #fit
        committee_member.train(Xtr, ytr)
        #predict
        ypool_lab.append(committee_member.model.predict(Xpool))
    #get probability of label for each class based on voting in the committee
    ypool_p = np.mean(np.array(ypool_lab), 0)
    #Refit model in all training data
    neural_net.fit(Xtrain,ytrain)
    testacc_qbc.append((len(Xtrain),neural_net.evaluate(Xtest, ytest)))
    #select sample with maximum disagreement (least confident)
    ypool_p_sort_idx = np.argsort(-ypool_p.max(1)) #least confident
    # make labels for new points
    ypool = None
    #add to training set
    Xtrain = np.concatenate((Xtrain,Xpool[ypool_p_sort_idx[-n_add:]]))
    ytrain = np.concatenate((ytrain,ypool[ypool_p_sort_idx[-n_add:]]))
    #remove from pool
    Xpool = np.delete(Xpool, ypool_p_sort_idx[-n_add:])
    #poolidx=np.setdiff1d(poolidx,poolidx[ypool_p_sort_idx[-n_add:]])
    print('Model: LR, %i samples (QBC)'%(n_samples+i*n_add))